# Content

explor how input text affect the output of the model,and
prompt engineering
- summarize dialogue without prompt engineering
- summarize dialogue with an instruction prompt

- comparing
  - zero shot inference
  - one shot inference
  - few shot inference


In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
  torch==1.13.1 \
  torchdata==0.5.1 --quiet

%pip install \
  transformers==4.27.2 \
  datasets==2.11.0 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 70.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

# Summarization without Prompt Engineering

generate summarization for Dialogue using pre-trained larg language model(flan-t5-base) from Hugging Face

let's upload some simple dialogue from the DialogSum Hugging Face dataset

In [3]:
Hugging_Face_dataset_name = "knkarthick/dialogsum"
data = load_dataset(Hugging_Face_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
example_indices = [40,200]
dash_line = '-'.join('' for x in range(100))


for i, index in enumerate(example_indices):
  print(dash_line)
  print(f'example {i+1}')
  print(dash_line)
  print(f'Input Dialogue')
  print(data['test'][index]['dialogue'])
  print(dash_line)
  print(f'Human Summary')
  print(data['test'][index]['summary'])
  print(dash_line)
  print("\n")




---------------------------------------------------------------------------------------------------
example 1
---------------------------------------------------------------------------------------------------
Input Dialogue
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Human Summary
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------


---------------------------------------------------------------------------------------------------
example 2
----

In [7]:
model_name = 'google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer= AutoTokenizer.from_pretrained(model_name  , use_fast =True)

test the tokenizer encode and decode

In [12]:
sentence = 'what time is it,Tom?'

sentence_encode = tokenizer(sentence , return_tensors='pt')
sentence_decode = tokenizer.decode( sentence_encode['input_ids'][0] , skip_special_tokens =True  )



print(f'encode sentence :')
print(sentence_encode['input_ids'][0])
print(dash_line)
print(f'decode sentence : \n{sentence_decode}')

encode sentence :
tensor([ 125,   97,   19,   34,    6, 3696,   51,   58,    1])
---------------------------------------------------------------------------------------------------
decode sentence : 
what time is it,Tom?


awesome let's start to generate summary for dialogue without using prompt engineering

In [14]:

for i,index in enumerate(example_indices):
  dialogue = data['test'][index]['dialogue']
  summary =  data['test'][index]['summary']
  input =    tokenizer(dialogue , return_tensors='pt')
  generate=  model.generate(input['input_ids'] , max_new_tokens=50)[0]
  output =   tokenizer.decode(generate , skip_special_tokens = True)

  print(dash_line)
  print(f'Example {i+1}')
  print(dash_line)
  print(f'the data dialogue:\n{dialogue}')
  print(dash_line)
  print(f'Human summary:\n{summary}')
  print(dash_line)
  print(f'Model Generation without any prompt Engineering')
  print(output)
  print(dash_line)
  print('\n')




---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
the data dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Human summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation without any prompt Engineering
Person1: It's ten to nine.
------------------------------------

# Zero Shot Inference With An Instruction Prompt

starting with zero shot inference which mean, we just take the dialogue and convert it into an instruction prompt to get good result

let's do it

In [16]:


for i, index in enumerate(example_indices):

  dialogue = data['test'][index]['dialogue']
  summary  = data['test'][index]['summary']
  prompt   = f'''
summarize the following conversation.

{dialogue}

summary:
  '''
  input    = tokenizer(prompt , return_tensors='pt')
  generate = model.generate(input['input_ids'] , max_new_tokens = 50 )[0]
  output   = tokenizer.decode(generate , skip_special_tokens=True)

  print(dash_line)
  print(f'Example {i+1}')
  print(dash_line)
  print(f'The Data Dialogue\n{dialogue}')
  print(dash_line)
  print(f'Human summary\n{summary}')
  print(dash_line)
  print(f'Model Generation Using Instruction Prompt Engineering "Zero Shot Inference" \n{output}')
  print(dash_line)








---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
The Data Dialogue
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Human summary
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation Using Instruction Prompt Engineering "Zero Shot Inference" 
The train is about to leave.
-------

this is much better, but model still does not up on the nuance of the conversation though

# Zero Shot Inference with the Prompt Template from FLAN-T5

Let's use a slightly different prompt. FLAN-T5 has many prompt templates that are published for certain tasks [here](https://github.com/google-research/FLAN/tree/main/flan/v2)

In [17]:


for i, index in enumerate(example_indices):

  dialogue = data['test'][index]['dialogue']
  summary  = data['test'][index]['summary']
  prompt   = f'''
Dialogue:

{dialogue}

what was going on?
  '''
  input    = tokenizer(prompt , return_tensors='pt')
  generate = model.generate(input['input_ids'] , max_new_tokens = 50 )[0]
  output   = tokenizer.decode(generate , skip_special_tokens=True)

  print(dash_line)
  print(f'Example {i+1}')
  print(dash_line)
  print(f'The Data Dialogue\n{dialogue}')
  print(dash_line)
  print(f'Human summary\n{summary}')
  print(dash_line)
  print(f'Model Generation Using Instruction Prompt Engineering "Zero Shot Inference" \n{output}')
  print(dash_line)








---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
The Data Dialogue
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Human summary
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation Using Instruction Prompt Engineering "Zero Shot Inference" 
Tom is late for the train.
---------

now it's the time for using few shot and one shot methods

# Using One Shot and Few Shot Inference

In [18]:
# let's create helpful function that make a prompt based on the  number of example that used.

def create_prompt(example_index , example_summary):
  prompt =''
  for index in(example_index):
    dialogue = data['test'][index]['dialogue']
    summary  = data['test'][index]['summary']
    prompt+= f"""
Dialogue:

{dialogue}

What was going on?
{summary}
    """

  sum_dialogue = data['test'][example_summary]['dialogue']
  sum_summary      = data['test'][example_summary]['summary']

  prompt+= f"""
Dialogue:

{sum_dialogue}

What was going on?
  """
  return prompt , sum_summary

# One Shot Inference

In [20]:
example_index   = [40]
example_summary = 200

one_shot_prompt , one_shot_summary = create_prompt(example_index , example_summary)

# generate summmary
one_shot_input    = tokenizer(one_shot_prompt , return_tensors='pt')
one_shot_generate = model.generate(one_shot_input['input_ids'] , max_new_tokens=50)[0]
one_shot_output   = tokenizer.decode(one_shot_generate , skip_special_tokens=True)


#let's show what we got

print(dash_line)
print(f'One Shot Inference:')
print(dash_line)
print(f'Prompt: \n{one_shot_prompt}')
print(dash_line)
print(f'Human Summary: \n{one_shot_summary}')
print(dash_line)
print(f'Model Generation Using One Shot: \n{one_shot_output}')


---------------------------------------------------------------------------------------------------
One Shot Inference:
---------------------------------------------------------------------------------------------------
Prompt: 
 
Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
     
Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: Th

# Few Shot Inference

In [22]:
example_index = [40, 80, 120]
example_summary = 200



few_shot_prompt , few_shot_summary = create_prompt(example_index , example_summary)

few_shot_input   = tokenizer(few_shot_prompt , return_tensors='pt')
few_shot_generate= model.generate(few_shot_input['input_ids'] , max_new_tokens=50)[0]
few_shot_output  = tokenizer.decode(few_shot_generate , skip_special_tokens=True)


print(dash_line)
print(f'Few Shot Inference:')
print(dash_line)
print(f'Prompt: \n{few_shot_prompt}')
print(dash_line)
print(f'Human Summary: \n{few_shot_summary}')
print(dash_line)
print(f'Model Generation Using Few Shot: \n{few_shot_output}')

---------------------------------------------------------------------------------------------------
Few Shot Inference:
---------------------------------------------------------------------------------------------------
Prompt: 
 
Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
     
Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No,

now we can say that the model ready to generate summary

in other hand we can add more conversations to test output like 5 or 6 dialogues

#Generative Configuration Parameters for Inference

In [24]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{few_shot_summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

